In [ ]:
from scipy.spatial import distance as dist
from imutils import perspective
from imutils import contours
import numpy as np
import argparse
import imutils
import cv2

def midpoint(ptA, ptB):
	return ((ptA[0] + ptB[0]) * 0.5, (ptA[1] + ptB[1]) * 0.5)

image = cv2.imread("F:\\internship\\weld bead\\13.jpg")
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
gray = cv2.GaussianBlur(gray, (7, 7), 0)
# perform edge detection, then perform a dilation + erosion to
# close gaps in between object edges
edged = cv2.Canny(gray, 50, 100)
edged = cv2.dilate(edged, None, iterations=1)
edged = cv2.erode(edged, None, iterations=1)
# find contours in the edge map
cnts = cv2.findContours(edged.copy(), cv2.RETR_EXTERNAL,
cv2.CHAIN_APPROX_SIMPLE)
cnts = imutils.grab_contours(cnts)
# sort the contours from left-to-right and initialize the
# 'pixels per metric' calibration variable
(cnts, _) = contours.sort_contours(cnts)
pixelsPerMetric = None

for c in cnts:
	# if the contour is not sufficiently large, ignore it
	if cv2.contourArea(c) < 5000:
		continue
	# compute the rotated bounding box of the contour
	orig = image.copy()
	box = cv2.minAreaRect(c)
	box = cv2.cv.BoxPoints(box) if imutils.is_cv2() else cv2.boxPoints(box)
	box = np.array(box, dtype="int")
	# order the points in the contour such that they appear
	# in top-left, top-right, bottom-right, and bottom-left
	# order, then draw the outline of the rotated bounding
	# box
	box = perspective.order_points(box)
	cv2.drawContours(orig, [box.astype("int")], -1, (0, 255, 0), 2)
	# loop over the original points and draw them
	for (x, y) in box:
		cv2.circle(orig, (int(x), int(y)), 5, (0, 0, 255), -1)
        
        
        # unpack the ordered bounding box, then compute the midpoint
	# between the top-left and top-right coordinates, followed by
	# the midpoint between bottom-left and bottom-right coordinates
	(tl, tr, br, bl) = box
	(tltrX, tltrY) = midpoint(tl, tr)
	(blbrX, blbrY) = midpoint(bl, br)
	# compute the midpoint between the top-left and top-right points,
	# followed by the midpoint between the top-righ and bottom-right
	(tlblX, tlblY) = midpoint(tl, bl)
	(trbrX, trbrY) = midpoint(tr, br)
	# draw the midpoints on the image
	cv2.circle(orig, (int(tltrX), int(tltrY)), 5, (255, 0, 0), -1)
	cv2.circle(orig, (int(blbrX), int(blbrY)), 5, (255, 0, 0), -1)
	cv2.circle(orig, (int(tlblX), int(tlblY)), 5, (255, 0, 0), -1)
	cv2.circle(orig, (int(trbrX), int(trbrY)), 5, (255, 0, 0), -1)
	# draw lines between the midpoints
	cv2.line(orig, (int(tltrX), int(tltrY)), (int(blbrX), int(blbrY)),
		(255, 0, 255), 2)
	cv2.line(orig, (int(tlblX), int(tlblY)), (int(trbrX), int(trbrY)),
		(255, 0, 255), 2)    

dA = dist.euclidean((tltrX, tltrY), (blbrX, blbrY))
dB = dist.euclidean((tlblX, tlblY), (trbrX, trbrY))
	# if the pixels per metric has not been initialized, then
	# compute it as the ratio of pixels to supplied metric
	# (in this case, inches)
if pixelsPerMetric is None:
	pixelsPerMetric = dB / 2

    
    
dimA = dA / pixelsPerMetric
dimB = dB / pixelsPerMetric
	# draw the object sizes on the image
cv2.putText(orig, "{:.1f}in".format(dimA),
    (int(tltrX - 15), int(tltrY - 10)), cv2.FONT_HERSHEY_SIMPLEX,
	0.65, (255, 255, 255), 2)
cv2.putText(orig, "{:.1f}in".format(dimB),
	(int(trbrX + 10), int(trbrY)), cv2.FONT_HERSHEY_SIMPLEX,
	0.65, (255, 255, 255), 2)
	# show the output image
orig = cv2.resize(orig, (600,400))    
cv2.imshow("Image", orig)
cv2.waitKey(0)    

In [2]:
pip install imutils

  Using cached imutils-0.5.4-py3-none-any.whl
Note: you may need to restart the kernel to use updated packages.


In [2]:
from scipy.spatial import distance as dist
from imutils import perspective
from imutils import contours
import numpy as np
import argparse
import imutils
import cv2
# construct the argument and parse command line input
#aparse = argparse.ArgumentParser()
#aparse.add_argument("--image", required=True,help="image path")
#aparse.add_argument("--width", type=float, required=True,help="width of far left object (inches)")
#args = vars(aparse.parse_args())
# load the image, convert it to grayscale, and blur it a bit
image = cv2.imread('F:\\internship\\weld bead\\13.jpg')
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
gray = cv2.GaussianBlur(gray, (7, 7), 0)

In [3]:
# perform edge detection + dilation + erosion to close gaps bt edges
edge_detect = cv2.Canny(gray, 15, 100) #play w/min and maxvalues to finetune edges (2nd n 3rd params)
edge_detect = cv2.dilate(edge_detect, None, iterations=1)
edge_detect = cv2.erode(edge_detect, None, iterations=1)

In [4]:
cntours = cv2.findContours(edge_detect.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
cntours = imutils.grab_contours(cntours)
# sort contours left-to-right
(cntours, _) = contours.sort_contours(cntours)
pixel_to_size = None
# function for finding the midpoint
def mdpt(A, B):
    return ((A[0] + B[0]) * 0.5, (A[1] + B[1]) * 0.5)

In [5]:
# loop over the contours individually
for c in cntours:
    if cv2.contourArea(c) < 100: #ignore/fly through contours that are not big enough
        continue
# compute the rotated bounding box of the contour; should handle cv2 or cv3..
orig = image.copy()
bbox = cv2.minAreaRect(c)
bbox = cv2.cv.boxPoints(bbox) if imutils.is_cv2() else cv2.boxPoints(bbox)
bbox = np.array(bbox, dtype="int")
# order the contours and draw bounding box
bbox = perspective.order_points(bbox)
cv2.drawContours(orig, [bbox.astype("int")], -1, (0, 255, 0), 2)

array([[[78, 87, 91],
        [77, 86, 90],
        [77, 86, 90],
        ...,
        [75, 86, 94],
        [75, 86, 94],
        [76, 87, 95]],

       [[78, 87, 91],
        [78, 87, 91],
        [78, 87, 91],
        ...,
        [76, 87, 95],
        [77, 88, 96],
        [77, 88, 96]],

       [[79, 88, 92],
        [79, 88, 92],
        [79, 88, 92],
        ...,
        [78, 89, 97],
        [78, 89, 97],
        [79, 90, 98]],

       ...,

       [[90, 84, 85],
        [89, 83, 84],
        [87, 81, 82],
        ...,
        [70, 64, 65],
        [70, 64, 65],
        [70, 64, 65]],

       [[90, 84, 85],
        [89, 83, 84],
        [87, 81, 82],
        ...,
        [70, 64, 65],
        [70, 64, 65],
        [70, 64, 65]],

       [[90, 84, 85],
        [89, 83, 84],
        [87, 81, 82],
        ...,
        [70, 64, 65],
        [70, 64, 65],
        [70, 64, 65]]], dtype=uint8)

In [6]:
# loop over the original points in bbox and draw them; 5pxred dots
for (x, y) in bbox:
    cv2.circle(orig, (int(x), int(y)), 5, (0, 0, 255),-1)
# unpack the ordered bounding bbox; find midpoints
    (tl, tr, br, bl) = bbox
    (tltrX, tltrY) = mdpt(tl, tr)
    (blbrX, blbrY) = mdpt(bl, br)
    (tlblX, tlblY) = mdpt(tl, bl)
    (trbrX, trbrY) = mdpt(tr, br)

In [7]:
# draw the mdpts on the image (blue);lines between the mdpts (yellow)
cv2.circle(orig, (int(tltrX), int(tltrY)), 5, (255, 0,0), -1)
cv2.circle(orig, (int(blbrX), int(blbrY)), 5, (255, 0,0), -1)
cv2.circle(orig, (int(tlblX), int(tlblY)), 5, (255, 0,0), -1)
cv2.circle(orig, (int(trbrX), in t(trbrY)), 5, (255, 0,0), -1)
cv2.line(orig, (int(tltrX), int(tltrY)), (int(blbrX),
int(blbrY)),(0, 255, 255), 2)
cv2.line(orig, (int(tlblX), int(tlblY)), (int(trbrX),
int(trbrY)),(0, 255, 255), 2)
# compute the Euclidean distances between the mdpts
dA = dist.euclidean((tltrX, tltrY), (blbrX, blbrY))
dB = dist.euclidean((tlblX, tlblY), (trbrX, trbrY))

In [8]:
# use pixel_to_size ratio to compute object size
if pixel_to_size is None:
    pixel_to_size = dB / 2000
    distA = dA / pixel_to_size
    distB = dB / pixel_to_size
# draw the object sizes on the image
cv2.putText(orig, "{:.1f}in".format(distA),
(int(tltrX - 10), int(tltrY - 10)), cv2.FONT_HERSHEY_DUPLEX,0.55, (255, 255, 255), 2)
cv2.putText(orig, "{:.1f}in".format(distB),
(int(trbrX + 10), int(trbrY)), cv2.FONT_HERSHEY_DUPLEX,0.55, (255, 255, 255), 2)
# show the output image
cv2.imshow("Image", orig)
cv2.waitKey(0)

-1

# Determining Size

In [4]:
from scipy.spatial.distance import euclidean
from imutils import perspective
from imutils import contours
import numpy as np
import imutils
import cv2

# Function to show array of images (intermediate results)
def show_images(images):
	for i, img in enumerate(images):
		cv2.imshow("image_" + str(i), img)
	cv2.waitKey(0)
	cv2.destroyAllWindows()

img_path = "images/example_02.jpg"

# Read image and preprocess
image = cv2.imread('F:\\internship\\weld bead\\13.jpg') 

gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
blur = cv2.GaussianBlur(gray, (9, 9), 0)

edged = cv2.Canny(blur, 50, 100)
edged = cv2.dilate(edged, None, iterations=1)
edged = cv2.erode(edged, None, iterations=1)

#show_images([blur, edged])

# Find contours
cnts = cv2.findContours(edged.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
cnts = imutils.grab_contours(cnts)

# Sort contours from left to right as leftmost contour is reference object
(cnts, _) = contours.sort_contours(cnts)

# Remove contours which are not large enough
cnts = [x for x in cnts if cv2.contourArea(x) > 100]

#cv2.drawContours(image, cnts, -1, (0,255,0), 3)

#show_images([image, edged])
#print(len(cnts))

# Reference object dimensions
# Here for reference I have used a 2cm x 2cm square
ref_object = cnts[0]
box = cv2.minAreaRect(ref_object)
box = cv2.boxPoints(box)
box = np.array(box, dtype="int")
box = perspective.order_points(box)
(tl, tr, br, bl) = box
dist_in_pixel = euclidean(tl, tr)
dist_in_mm = 5
pixel_per_mm = dist_in_pixel/dist_in_mm

# Draw remaining contours
for cnt in cnts:
	box = cv2.minAreaRect(cnt)
	box = cv2.boxPoints(box)
	box = np.array(box, dtype="int")
	box = perspective.order_points(box)
	(tl, tr, br, bl) = box
	cv2.drawContours(image, [box.astype("int")], -1, (0, 0, 255), 2)
	mid_pt_horizontal = (tl[0] + int(abs(tr[0] - tl[0])/2), tl[1] + int(abs(tr[1] - tl[1])/2))
	mid_pt_verticle = (tr[0] + int(abs(tr[0] - br[0])/2), tr[1] + int(abs(tr[1] - br[1])/2))
	wid = euclidean(tl, tr)/pixel_per_mm
	ht = euclidean(tr, br)/pixel_per_mm
	cv2.putText(image, "{:.1f}mm".format(wid), (int(mid_pt_horizontal[0] - 15), int(mid_pt_horizontal[1] - 10)), 
		cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 0), 2)
	cv2.putText(image, "{:.1f}mm".format(ht), (int(mid_pt_verticle[0] + 10), int(mid_pt_verticle[1])), 
		cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 0), 2)
image = cv2.resize(image, (400,400))
show_images([image])